In [1]:
import pandas as pd
import requests
#Function that returns income statement data
def get_data2(stock):
    data = requests.get(f"https://financialmodelingprep.com/api/v3/income-statement/{stock}?period=quarter&limit=400&apikey=fmp")

    data = data.json()
# Pull data from the previous for quarters. The 0 refers to the most # previous quarter etc. 
    data = data[0:5]
    
    return data

In [2]:
def growth(stock):
    #call the get_data2 function and store the results in a       #variable, data 2.
    data = get_data2(stock)
    
    #revenue growth
    rev_new = data[0]['revenue']
    rev_old = data[-1]['revenue']
    rev_change = 100*((rev_new-rev_old)/rev_old)
    #net income growth
    income_new = data[0]['netIncome']
    income_old = data[-1]['netIncome']
    income_change = 100*((income_new-income_old)/income_old)
    
    #EPS growth
    eps_new = data[0]['eps']
    eps_old = data[-1]['eps']
    eps_change = 100*((eps_new-eps_old)/eps_old)
    
    #EBITA
    ebitda_new = data[0]['ebitda']
    ebitda_old = data[-1]['ebitda']
    ebitda_change = 100*((ebitda_new-ebitda_old)/ebitda_old)
    
    return (rev_change, income_change, eps_change, ebitda_change)

In [3]:
stocks = ['PTON', 'MSFT', 'AAPL', 'AMD']
#store financials in a variable, data.
data = map(growth, stocks)
# populate DataFrame with data variable and create columns:
df = pd.DataFrame(data, columns=['Rev_Growth', 'Income_Growth', 'EPS_Growth', 'ebitda_Growth'], index=stocks)
df.reset_index(inplace=True)
df.rename(columns={'index': 'Symbol'}, inplace=True)
#display output
df

TypeError: unhashable type: 'slice'

In [ ]:
def dcf(stock):
    data = requests.get(f"https://financialmodelingprep.com/api/v3/discounted-cash-flow/{stock}?apikey=fmp")
    data = data.json()
    dcf = data[0]['dcf']
    current_price = data[0]['Stock Price']
    return (dcf, current_price)

In [ ]:
stocks = ['PTON', 'MSFT', 'AAPL', 'AMD']
data = map(dcf, stocks)
df = pd.DataFrame(data, columns=['dcf', 'current price'], index=stocks)
df.reset_index(inplace=True)
df.rename(columns={'index': 'symbol'}, inplace=True)
#Create a column that indicates under or over-valued
df['value'] = df.apply(lambda x: 'Under-valued' if x['dcf']>x['current price'] else 'Over-valued',axis=1)
#display results
df